# APC-CLI 基本チュートリアル 実行フロー

基本的に各ステップを上から順に実行することで、AutoPrivacy DCR上で関数を実行することができます。
AutoPrivacy DCRの操作をCLIから行うツールとして、APC-CLIというコマンドラインツールを用います。

**このJupyterNotebookは[こちら](https://github.com/acompany-develop/dcr-docs-examples/blob/main/notebooks/tutorial/basic_apc_cli_tutorial.ipynb)に上がっているため、ステップ2の環境変数さえ設定すれば、あとは説明に従って各セルを実行するだけで、AutoPrivacy DCR上で処理を実行できるようになっています。**

## 概要

このスクリプトは以下の主要なステップで構成されています：
1. 全体概要
2. 環境変数設定
3. APC-CLI のセットアップ
4. プロファイル設定
5. 入出力の設定ファイルの生成
6. 認証とヘルスチェック
7. プロジェクト設定
8. 関数の準備
9. 関数ストレージへのアップロード
10. Cleanroom デプロイ
11. データの Cleanroom へのコピー
12. Cleanroom の実行
13. 結果のダウンロード
14. クリーンアップ

-------------

## 1. 全体概要
Cleanroomを用いることで、二者間で安全にデータを共有し実行することが可能となります。APC-CLI実行時に基本的にprofile（user）を指定するのですが、profileが異なれば異なるユーザーから実行されることを想定しているため、異なるマシンから実行することが可能です（同一のマシンから実行することも可能）。
具体的なフロー図は以下のようになります。

<details><summary>フロー図</summary>
<img src="../img/apc_sequence.svg" width="800">
</details>

-------------

## 2. 環境変数設定
ファイル・ディレクトリ構成としては、今回は以下のようなものを想定します。この詳細については[こちら](https://acompany-develop.github.io/autoprivacy-cloud/apc-dcr/user-guide/user-files/function-directory.html)から参照できます。

```
FUNCTION_SOURCE_PATH
├── function # 関数のパス
│   └── handler.py
│   └── packages # 依存パッケージのパス
├── inputs # 入力データのパス
│   ├── input_1
│   └── input_2
└── outputs # 出力データのパス
    ├── output_1
    └── output_2
```

このチュートリアルではjoin関数をCleanroom上で実行します。join関数の全実装は[こちら](https://github.com/acompany-develop/dcr-docs-examples/blob/main/functions/join/function/handler.py)のようになります。

実行に必要な環境変数を.envファイルに書き込みます。`API_URL`, `ATTESTATION_URL`, `ATTESTATION_API_VERSION`, `MR_ENCLAVE`, `MR_SIGNER`, `CLIENT_ID1`, `CLIENT_SECRET1`, `CLIENT_ID2`, `CLIENT_SECRET2`, `PROJECT_ID`という環境変数の値はAutoPrivacy DCRサービス提供者から提供されたものを使用します。ただし、ダブルクオーテーションはつけずに値だけを環境変数に代入してください。

なお、データ共有を行う二者間では、`CLIENT_ID`と`CLIENT_SECRET`だけ異なり、それ以外環境変数の値（`API_URL`, `ATTESTATION_URL`, `ATTESTATION_API_VERSION`, `MR_ENCLAVE`, `MR_SIGNER`, `PROJECT_ID`）は同一になります。

In [ ]:
# マシンに保存される設定に紐づく識別子。ユーザーが自由に設定することが出来る。
%env profile1=user1
# マシンに保存される設定に紐づく識別子。ユーザーが自由に設定することが出来る。
%env profile2=user2

%env API_URL=
%env ATTESTATION_URL=
%env ATTESTATION_API_VERSION=
%env MR_ENCLAVE=
%env MR_SIGNER=
%env CLIENT_ID1=
%env CLIENT_SECRET1=
%env CLIENT_ID2=
%env CLIENT_SECRET2=
%env PROJECT_ID=

# 入出力設定ファイルのパス
%env ENCRYPTED_FILES_PATH=encrypted-files.yaml

-------------

## 3. APC-CLI のセットアップ

詳細なAPC-CLIのセットアップ方法については、[こちら](https://acompany-develop.github.io/autoprivacy-cloud/apc-cli/getting-started/installation.html)から参照できます。

まず、`apc_cli_version`は[こちら](https://github.com/acompany-develop/apc-cli/releases)を参照し、APC-CLIのバージョンを指定してください。以下は例として、`2.0.0`を指定しています。

In [ ]:
%env APC_CLI_VERSION=2.0.0

指定したバージョンでAPC-CLIをダウンロードします。

In [ ]:
%%bash

# OSを判定
OS=$(uname -s)

case $OS in
    "Darwin")
        # macOS
        echo "macOS detected"
        wget https://github.com/acompany-develop/apc-cli/releases/download/$APC_CLI_VERSION/apc-darwin-arm64.zip
        unzip apc-darwin-arm64.zip
        mv apc-darwin-arm64 apc
        chmod +x apc
        rm apc-darwin-arm64.zip
        ;;
    "Linux")
        # Linux
        echo "Linux detected"
        wget https://github.com/acompany-develop/apc-cli/releases/download/$APC_CLI_VERSION/apc-linux-x64.zip
        unzip apc-linux-x64.zip
        mv apc-linux-x64 apc
        chmod +x apc
        rm apc-linux-x64.zip
        ;;
    *)
        echo "Unknown OS: $OS"
        exit 1
        ;;
esac

ダウンロードしたAPC-CLIのパスを通します。

In [ ]:
import os

os.environ["PATH"] = os.path.dirname(os.path.abspath("apc")) + ":" + os.environ["PATH"]

APC-CLIが正しくインストールされているか確認します。

In [ ]:
%%bash
apc --version

-------------

## 4. プロファイル設定
各プロファイルに対して種設定情報を入力し、User IDを生成します。
今回のチュートリアルでは標準出力されるUserIDを環境変数に読み込むために以下のようにコマンドを実行します。
コマンドの詳細は [configure コマンド リファレンス](https://acompany-develop.github.io/autoprivacy-cloud/apc-cli/commands/configure.html) から参照できます。

まずは、`profile1`について設定します。出力された`USER_ID`は次の入出力設定のステップで使用します。

In [ ]:
%%bash
expect <<EOF
spawn apc configure --profile $profile1
expect "API URL: "
send "$API_URL\r"
expect "Attestation URL: "
send "$ATTESTATION_URL\r"
expect "Attestation API Version: "
send "$ATTESTATION_API_VERSION\r"
expect "Attestation MR Enclave: "
send "$MR_ENCLAVE\r"
expect "Attestation MR Signer: "
send "$MR_SIGNER\r"
expect "Client ID: "
send "$CLIENT_ID1\r"
expect "Client Secret: "
send "$CLIENT_SECRET1\r"
expect eof
EOF

次に、`profile2`について設定します。こちらも同様に、出力された`USER_ID`は次の入出力設定のステップで使用します。

In [ ]:
%%bash
expect <<EOF
spawn apc configure --profile $profile2
expect "API URL: "
send "$API_URL\r"
expect "Attestation URL: "
send "$ATTESTATION_URL\r"
expect "Attestation API Version: "
send "$ATTESTATION_API_VERSION\r"
expect "Attestation MR Enclave: "
send "$MR_ENCLAVE\r"
expect "Attestation MR Signer: "
send "$MR_SIGNER\r"
expect "Client ID: "
send "$CLIENT_ID2\r"
expect "Client Secret: "
send "$CLIENT_SECRET2\r"
expect eof
EOF

-------------

## 5. 入出力の設定ファイルの生成

入出力の設定ファイルを`$ENCRYPTED_FILES_PATH`に作成します。詳細は[こちら](https://acompany-develop.github.io/autoprivacy-cloud/apc-dcr/user-guide/user-files/definition-files.html)から参照できます。

**ただし、実行前に**`<USER_1_ID>`**と**`<USER_2_ID>`**を5で出力された**`profile1`**と**`profile2`**のUser IDに置き換えてください。**

In [ ]:
%%bash
cat > $ENCRYPTED_FILES_PATH << EOF
inputs:
  input_1: &user_1_id <USER_1_ID>
  input_2: &user_2_id <USER_2_ID>
outputs:
  output_1: *user_1_id
  output_2: *user_2_id
EOF
cat $ENCRYPTED_FILES_PATH

-------------

## 6. 認証とヘルスチェック

各プロファイルでログインします。コマンドの詳細は [auth-login コマンド リファレンス](https://acompany-develop.github.io/autoprivacy-cloud/apc-cli/commands/auth-login.html) から参照できます。

In [ ]:
%%bash
apc auth-login --profile $profile1
apc auth-login --profile $profile2

各プロファイルで、サーバーに対してヘルスチェックを行い、正常にサーバーと通信ができているかを確認します。コマンドの詳細は [healthcheck コマンド リファレンス](https://acompany-develop.github.io/autoprivacy-cloud/apc-cli/commands/healthcheck.html) から参照できます。

In [ ]:
%%bash
apc healthcheck --profile $profile1
apc healthcheck --profile $profile2

-------------

## 7. プロジェクト設定

各プロファイルで同じプロジェクトを設定します。コマンドの詳細は [set-project コマンド リファレンス](https://acompany-develop.github.io/autoprivacy-cloud/apc-cli/commands/set-project.html) から参照できます。

In [ ]:
%%bash
apc set-project $PROJECT_ID --profile $profile1
apc set-project $PROJECT_ID --profile $profile2

-------------

## 8. 関数の準備
Cleanroom上で実行する関数のパスを設定します。
今回のチュートリアルでは、リポジトリの中のjoin関数のパスを`FUNCTION_SOURCE_PATH`に指定します。
なお、[このディレクトリ](https://github.com/acompany-develop/dcr-docs-examples/tree/main/functions)の中の関数はCleanroom上で実行可能であり、これらの関数のパスを指定することも可能です。

In [ ]:
import os

%env FUNCTION_SOURCE_PATH=../../functions/cross_table

FUNCTION_SOURCE_PATH = os.environ.get("FUNCTION_SOURCE_PATH")

%env FUNCTION_SOURCE_PATH={FUNCTION_SOURCE_PATH}
# 関数のパス
%env FUNCTION_DIRECTORY_PATH={FUNCTION_SOURCE_PATH}/function
# 入力データのパス
%env INPUT_1_PATH={FUNCTION_SOURCE_PATH}/inputs/input_1
%env INPUT_2_PATH={FUNCTION_SOURCE_PATH}/inputs/input_2
# 出力データのパス
%env OUTPUT_1_PATH={FUNCTION_SOURCE_PATH}/outputs/output_1
%env OUTPUT_2_PATH={FUNCTION_SOURCE_PATH}/outputs/output_2

Cleanroom上での関数実行で使用するライブラリを`packages`ディレクトリにインストールします。

In [ ]:
%%bash
pip install --platform manylinux2014_x86_64 \
    --only-binary=:all: \
    --python-version 3.10  \
    --target=$FUNCTION_DIRECTORY_PATH/packages \
    -r $FUNCTION_SOURCE_PATH/requirements.txt

-------------

## 9. 関数ストレージへのアップロード

関数ディレクトリのパスを指定した上で、実行する関数をCleanroom上にアップロードします。コマンドの詳細は [function-storage コマンド リファレンス](https://acompany-develop.github.io/autoprivacy-cloud/apc-cli/commands/function-storage.html#function-storage-upload) から参照できます。なお、出力された`FunctionStoragePath`は次のCleanroomデプロイのステップで使用します。

In [ ]:
%%bash
apc function-storage upload --source $FUNCTION_DIRECTORY_PATH --profile $profile1

-------------

## 10. Cleanroom デプロイ

アップロードされた関数を使用してCleanroomアプリケーションをデプロイします。
コマンドの詳細は [cleanroom deploy コマンド リファレンス](https://acompany-develop.github.io/autoprivacy-cloud/apc-cli/commands/cleanroom.html#cleanroom-deploy) から参照できます。

**ただし、実行前に**`--source`**オプションの引数の<FUNCTION_STORAGE_PATH>を9で出力された**`FunctionStoragePath`**の値に置き換えてください。**

In [ ]:
%%bash
apc cleanroom deploy \
    --source <FUNCTION_STORAGE_PATH> \
    --name join_app \
    --runtime python3.10 \
    --handler handler.run \
    --encrypted-files $ENCRYPTED_FILES_PATH \
    --memory 2 \
    --profile $profile1

-------------

## 11. データの Cleanroom へのコピー

各プロファイルから、ローカルの入力データをCleanroom上にコピーします。コマンドの詳細は [cleanroom copyコマンド リファレンス](https://acompany-develop.github.io/autoprivacy-cloud/apc-cli/commands/cleanroom-data.html) から参照できます。

In [ ]:
%%bash
apc cleanroom data cp $INPUT_1_PATH join_app:input_1 --profile $profile1
apc cleanroom data cp $INPUT_2_PATH join_app:input_2 --profile $profile2

-------------

## 12. Cleanroom の実行

デプロイされたCleanroom上のアプリケーションを実行します。コマンドの詳細は [cleanroom run コマンド リファレンス](https://acompany-develop.github.io/autoprivacy-cloud/apc-cli/commands/cleanroom.html#cleanroom-run) から参照できます。

In [ ]:
%%bash
apc cleanroom run join_app --profile $profile1

-------------

## 13. 結果のダウンロード

実行結果を各プロファイルにダウンロードします。コマンドの詳細は [cleanroom data コマンド リファレンス](https://acompany-develop.github.io/autoprivacy-cloud/apc-cli/commands/cleanroom-data.html) から参照できます。

In [ ]:
%%bash
apc cleanroom data cp join_app:output_1 $OUTPUT_1_PATH --profile $profile1
apc cleanroom data cp join_app:output_2 $OUTPUT_2_PATH --profile $profile2

-------------

## 14. クリーンアップ

使用したリソースに変更を加える必要があり、削除したい場合はリソースをクリーンアップすることができます。

Cleanroom上のアプリケーションを削除します。コマンドの詳細は [cleanroom delete コマンド リファレンス](https://acompany-develop.github.io/autoprivacy-cloud/apc-cli/commands/cleanroom.html#cleanroom-delete) から参照できます。

In [ ]:
%%bash
apc cleanroom delete join_app --profile $profile1

Cleanroom上にアップロードされた関数を削除します。コマンドの詳細は [function-storage delete コマンド リファレンス](https://acompany-develop.github.io/autoprivacy-cloud/apc-cli/commands/function-storage.html#function-storage-delete) から参照できます。

**ただし、実行前に<FUNCTION_STORAGE_PATH>を9で出力された`FunctionStoragePath`の値に置き換えてください。**

In [ ]:
%%bash
apc function-storage delete <FUNCTION_STORAGE_PATH> --profile $profile1